# Analyze eBPF Programs

This notebook compiles, analyzes, and generates the report for eBPF programs. 

The raw report is saved in the `output/report` directory.

In [1]:
%reload_ext autoreload
%autoreload 2

The following cell compiles the BCC tools. 

In [2]:
from utils import BCC_TOOLS_PATH

!make -C {BCC_TOOLS_PATH} -j $(nproc)

make: Entering directory '/users/szhong/DepSurf/software/bcc/libbpf-tools'
  CC       profile.o
  BPF      readahead.bpf.o
  BINARY   profile
  GEN-SKEL readahead.skel.h
  CC       readahead.o
  BINARY   readahead
make: Leaving directory '/users/szhong/DepSurf/software/bcc/libbpf-tools'


The following cell compiles the eBPF program for Tracee.

In [3]:
from utils import TRACEE_PATH

!make -C {TRACEE_PATH} bpf

make: Entering directory '/users/szhong/DepSurf/software/tracee'
make: Nothing to be done for 'bpf'.
make: Leaving directory '/users/szhong/DepSurf/software/tracee'


The following cell analyzes and generates the report for eBPF programs.

In [3]:
from depsurf import BPFProgram, VersionGroup, gen_report, DepKind
from utils import TRACEE_PATH, BCC_OBJ_PATH, OUTPUT_PATH, save_pkl
import logging

REPORT_PATH = OUTPUT_PATH / "report"
REPORT_PATH.mkdir(parents=True, exist_ok=True)
GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH]
PROG_PATHS = list(BCC_OBJ_PATH.glob("*.bpf.o")) + [TRACEE_PATH / "dist" / "tracee.bpf.o"]

KINDS = [
    DepKind.FUNC,
    DepKind.TRACEPOINT,
    DepKind.LSM,
    DepKind.FIELD,
    DepKind.STRUCT,
    DepKind.SYSCALL,
]

results = {}
for path in PROG_PATHS:
    bpf = BPFProgram(path)
    result_path = REPORT_PATH / f"{bpf.name}.log"
    with open(result_path, "w") as f:
        results[bpf.name] = {dep : gen_report(dep, GROUPS, file=f) for dep in bpf.deps if dep.kind in KINDS}
        logging.info(f"Report saved to {result_path}")


save_pkl(results, "bcc")

[         dump.py:161] INFO: Normalizing types from /Users/szhong/Code/DepSurf/software/bcc/libbpf-tools/.output/hardirqs.bpf.min.btf.json
[   1081259294.py:25 ] INFO: Report saved to /Users/szhong/Code/DepSurf/output/report/hardirqs.log
[         dump.py:161] INFO: Normalizing types from /Users/szhong/Code/DepSurf/software/bcc/libbpf-tools/.output/filelife.bpf.min.btf.json
[   1081259294.py:25 ] INFO: Report saved to /Users/szhong/Code/DepSurf/output/report/filelife.log
[         dump.py:161] INFO: Normalizing types from /Users/szhong/Code/DepSurf/software/bcc/libbpf-tools/.output/oomkill.bpf.min.btf.json
[   1081259294.py:25 ] INFO: Report saved to /Users/szhong/Code/DepSurf/output/report/oomkill.log
[         dump.py:161] INFO: Normalizing types from /Users/szhong/Code/DepSurf/software/bcc/libbpf-tools/.output/opensnoop.bpf.min.btf.json
[   1081259294.py:25 ] INFO: Report saved to /Users/szhong/Code/DepSurf/output/report/opensnoop.log
[         dump.py:161] INFO: Normalizing types f